In [ ]:
# Dartmouth and Columbia
# in this notebook there are 6 dfs:
# Dartmouth men, women, combined mens and womens
# Columbia men, women, combined mens and womens

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import matplotlib.pyplot as plt
import pandas as pd
import time
import re

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver

from IPython.display import display, Image

In [97]:
# function to extract dataframe from website
def sports_df(link):

    year_list = []
    for num in range(2010,2024):
        year_list.append(num)

    url_list = []
    base_url = link
    for yr in year_list:
        new_url = f"{base_url}/{yr}"
        url_list.append(new_url)

    df_list = []
    for url in url_list:
        data = pd.read_html(url)[2]
        df_list.append(data)
        
    #adding year column
    for year_num in range(len(year_list)):
        df_list[year_num]["year"] = year_list[year_num]

    #rosters from older years like 2010 have different column names than the more recent years, so I
    #just renamed selected columns to match
    new_dfs = []
    for df in df_list:
        new_dfs.append(df.rename(columns = {"Full Name" : "Name",
                            "Academic Year" : "Class",
                            "Pos" : "Pos.",
                            "Yr." : "Class",
                            "Year" : "Class",               
                            "Event" : "Pos.",
                            "Hometown / Previous School" : "Hometown / High School",
                            "Hometown / Previous School(s)" : "Hometown / High School",                
                            "Previous School": "Previous Club"}))
            
    football_df = pd.concat(new_dfs, join = "outer", ignore_index = True)
    if "#" in football_df.columns:
        r = football_df.drop(columns = "#")
        return r
    else:
        return football_df
    
# function for columbia, splits the 'Hometown / High School' into 2 seperate columns
def split(df):
    data = df
    if "Hometown / Previous School(s)" in data.columns:
        data[['Hometown', 'High School']] = data["Hometown / Previous School(s)"].str.split('/', expand=True)
        return data
    else:
        f = data["Hometown / High School"].str.split('/', expand=True)
        if 2 in f.columns:
            new_df = f.drop(columns = 2)
            data[["Hometown", "High School"]] = new_df
            return data
        data[["Hometown", "High School"]] = data["Hometown / High School"].str.split('/', expand=True)
        return data
            

In [111]:
#Dartmouth Mens
# the 'C' column is from soccer, idk what it measures and you can drop it if you'd like
sports = ["football", "basketball","track","soccer"]
sports_url = ["https://dartmouthsports.com/sports/football/roster",
             "https://dartmouthsports.com/sports/mens-basketball/roster",
             "https://dartmouthsports.com/sports/mens-track-and-field/roster",
             "https://dartmouthsports.com/sports/mens-soccer/roster"]
mens = []
for url in sports_url:
    df = sports_df(url)
    mens.append(df)

for num in range(len(sports)):
    mens[num]["sport"] = sports[num]
    
mens_df = pd.concat(mens, join = "outer", ignore_index = True).drop(columns = "Hometown / High School")
mens_df

,Name,Pos.,Ht.,Wt.,Class,High School,Hometown,year,sport,C
0,Pat Scorah,LB,5-8,210.0,Sr.,St. John's College HS,"Washington, D.C.",2010,football,NaN
1,Tim McManus,WR,6-0,195.0,Sr.,St. Thomas Academy,"St. Paul, Minn.",2010,football,NaN
2,Austin Katigan,WR,5-9,170.0,Fy.,Casady,"Oklahoma City, Okla.",2010,football,NaN
3,Donald Kephart,K,5-11,220.0,Sr.,Acalanes,"Lafayette, Calif.",2010,football,NaN
4,T.J. Cameron,RB,5-11,210.0,Sr.,Greenwich,"Greenwich, Conn.",2010,football,NaN
...,...,...,...,...,...,...,...,...,...,...
2817,Raul Vargas,M,5-8,147.0,So.,Green Valley Atenas,"Atenas, Costa Rica",2023,soccer,NaN
2818,Orrett Maine,M,6-3,185.0,Sr.,Greater Atlanta Christian,"Atlanta, Ga.",2023,soccer,NaN
2819,David Mysiuk,D,6-0,165.0,Jr.,Bradley-Bourbonnais,"Bourbonnais, Ill.",2023,soccer,NaN
2820,Jonathan Preston,M,5-7,155.0,Jr.,Shattuck St. Mary's Academy,"Brentwood, Tenn.",2023,soccer,NaN


In [110]:
#Dartmouth Womens
# Volleyball is not included bc I had trouble cleaning that dataset (see below)
w_sports = ["basketball", "track","soccer"]
w_sports_url = ["https://dartmouthsports.com/sports/womens-basketball/roster",
             "https://dartmouthsports.com/sports/womens-track-and-field/roster",
             "https://dartmouthsports.com/sports/womens-soccer/roster"]

womens = []
for url in w_sports_url:
    df = sports_df(url)
    womens.append(df)

for num in range(len(w_sports)):
    womens[num]["sport"] = w_sports[num]
    
womens_df = pd.concat(womens, join = "outer", ignore_index = True)
womens_df

,Name,Pos.,Ht.,Class,High School,Hometown,year,sport,C
0,Nina Minicozzi,G,5-8,Fy.,The Rivers School,"Belmont, Mass.",2010,basketball,NaN
1,Meena Tate,G,5-11,Fy.,Iowa City West,"Iowa City, Iowa",2010,basketball,NaN
2,Mekkena Boyd,G,5-8,Sr.,Champlain Valley Union,"Williston, Vt.",2010,basketball,NaN
3,Cate MacDonald,F,6-1,Fy.,New Hampton,"Lynnfield, Mass.",2010,basketball,NaN
4,Mia Curtis,G,5-9,Sr.,Minnehaha Academy,"Minneapolis, Minn.",2010,basketball,NaN
...,...,...,...,...,...,...,...,...,...
1295,Hannah Curtin,F,5-6,Jr.,Winchester,"Winchester, Mass.",2023,soccer,NaN
1296,Ellie Davidson,D,5-4,Fy.,Torrey Pines,"San Diego, Calif.",2023,soccer,NaN
1297,Kate Ryan,D,5-7,Fy.,Ursuline Academy of Dallas,"Dallas, Texas",2023,soccer,NaN
1298,Ola Goebel,GK,5-7,Fy.,Evanston Township,"Chicago, Ill.",2023,soccer,NaN


In [22]:
#Dartmouth Women's Volleyball
# the earlier years have high school and hometown in one column whereas the later years have seperate
#columns for high school and hometown. I was trying to sepereate the hometown / highschool column and replace
# the NaNs with the newly seperate columns but I couldn't figure it out :( 
sports_df("https://dartmouthsports.com/sports/womens-volleyball/roster")

,Name,Pos.,Ht.,Class,Hometown / High School,year,High School,Hometown
0,Katelyn Rogers,MB,-,Fy.,"Simi Valley, Calif. / Chaminade College Prep",2010,NaN,NaN
1,Rachel Aragon,OH,-,Fy.,"Whittier, Calif. / Mayfield School",2010,NaN,NaN
2,Alissa Santa Maria,S,5-10,So.,"Yorba Linda, Calif. / El Dorado",2010,NaN,NaN
3,Lexi Campbell,OH/DS,5-10,So.,"Solana Beach, Calif. / Torrey Pines",2010,NaN,NaN
4,Kendall Houston,S,5-9,Jr.,"Austin, Texas / Stephen F. Austin",2010,NaN,NaN
...,...,...,...,...,...,...,...,...
245,Maddie LaFata,RS,5-11,Fy.,NaN,2023,Fayetteville,"Fayetteville, Ark."
246,Ava Roberts,MB/RS,6-0,So.,NaN,2023,Mansfield,"Mansfield, Texas"
247,Bomi Ogunlari,MB/RS,5-9,Sr.,NaN,2023,Rockwall,"Rockwall, Texas"
248,Raeli Smith,MB,6-2,Fy.,NaN,2023,Centennial,"Peoria, Ariz."


In [109]:
#Dartmouth men and womens
mens_df["sex"] = "mens"
womens_df["sex"] = "womens"

both = [mens_df, womens_df]
pd.concat(both, join = "outer", ignore_index = True)

,Name,Pos.,Ht.,Wt.,Class,High School,Hometown,year,sport,C,sex
0,Pat Scorah,LB,5-8,210.0,Sr.,St. John's College HS,"Washington, D.C.",2010,football,NaN,mens
1,Tim McManus,WR,6-0,195.0,Sr.,St. Thomas Academy,"St. Paul, Minn.",2010,football,NaN,mens
2,Austin Katigan,WR,5-9,170.0,Fy.,Casady,"Oklahoma City, Okla.",2010,football,NaN,mens
3,Donald Kephart,K,5-11,220.0,Sr.,Acalanes,"Lafayette, Calif.",2010,football,NaN,mens
4,T.J. Cameron,RB,5-11,210.0,Sr.,Greenwich,"Greenwich, Conn.",2010,football,NaN,mens
...,...,...,...,...,...,...,...,...,...,...,...
4117,Hannah Curtin,F,5-6,NaN,Jr.,Winchester,"Winchester, Mass.",2023,soccer,NaN,womens
4118,Ellie Davidson,D,5-4,NaN,Fy.,Torrey Pines,"San Diego, Calif.",2023,soccer,NaN,womens
4119,Kate Ryan,D,5-7,NaN,Fy.,Ursuline Academy of Dallas,"Dallas, Texas",2023,soccer,NaN,womens
4120,Ola Goebel,GK,5-7,NaN,Fy.,Evanston Township,"Chicago, Ill.",2023,soccer,NaN,womens


In [106]:
#Columbia mens
#Previous Club column is from soccer, Event is from track

c_sports = ["football", "basketball","soccer"]
c_sports_url = ["https://gocolumbialions.com/sports/football/roster",
             "https://gocolumbialions.com/sports/mens-basketball/roster",
             "https://gocolumbialions.com/sports/mens-soccer/roster"]
c_mens = []
for url in c_sports_url:
        df = split(sports_df(url)).drop(columns = "Hometown / High School")
        c_mens.append(df)

for num in range(len(c_sports)):
    c_mens[num]["sport"] = c_sports[num]
    
# track & field has a different page layout so had to webscrape manually
year_list = []
for num in range(2010,2024):
    year_list.append(num)

url_list = []
base_url = "https://gocolumbialions.com/sports/track-and-field/roster"
for yr in year_list:
    new_url = f"{base_url}/{yr}"
    url_list.append(new_url)
    

df_list = []
for url in url_list:
    data = pd.read_html(url)[3]
    df_list.append(data.rename(columns = {"Full Name": "Name",
                                         "Year": "Class"}))
    
for year_num in range(len(year_list)):
    df_list[year_num]["year"] = year_list[year_num]

track = split(pd.concat(df_list, join = "outer")).drop(columns = "Hometown / High School")
track["sport"] = "track"
c_mens.append(track)
    
c_mens_df = pd.concat(c_mens, join = "outer", ignore_index = True)
c_mens_df

,Name,Pos.,Ht.,Class,year,Wt.,School,Hometown,High School,sport,Previous Club,Event
0,Paul Delaney,P,6-3,FY,2010,NaN,NaN,"Palatine, Ill.",Loyola Academy,football,NaN,NaN
1,Mike Stephens,WR,5-11,Sr.,2010,NaN,NaN,"Flower Mound, Texas",Flower Mound,football,NaN,NaN
2,Brian DeVeau,WR,5-11,So.,2010,NaN,NaN,"Kinnelon, N.J.",Kinnelon,football,NaN,NaN
3,Nico Gutierrez,WR,6-0,Sr.,2010,NaN,NaN,"New Canaan, Conn.",New Canaan,football,NaN,NaN
4,Ryan Murphy,LB,6-2,So.,2010,NaN,NaN,"San Antonio, Texas",Alamo Heights,football,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2827,Jacob Van Orden,NaN,NaN,So.,2023,NaN,CC,"Pocatello, Id.",Highland,track,NaN,Middle Distance
2828,Gavin White,NaN,NaN,Jr.,2023,NaN,CC,"Pinckney, Mich.",Pinckney Community,track,NaN,Distance
2829,Zach Willen,NaN,NaN,FY,2023,NaN,CC,"Lansdale, Pa.",Methacton,track,NaN,Sprints
2830,Josiah Williams,NaN,NaN,So.,2023,NaN,CC,"Clayton, N.C.",Clayton,track,NaN,Sprints


In [112]:
#Columbia womens
c_w_sports = ["basketball","volleyball","track"]
c_w_sports_url = ["https://gocolumbialions.com/sports/womens-basketball/roster",
             "https://gocolumbialions.com/sports/womens-volleyball/roster",
                 "https://gocolumbialions.com/sports/track-and-field/roster"]
c_womens = []
for url in c_w_sports_url:
    df = split(sports_df(url)).drop(columns = "Hometown / High School")
    c_womens.append(df)
    
for num in range(len(c_w_sports)):
    c_womens[num]["sport"] = c_w_sports[num]
    
#soccer is missing 2021 data so had to manually scrape it
year_list = []
for num in range(2010,2024):
    year_list.append(num)

url_list = []
base_url = "https://gocolumbialions.com/sports/womens-soccer/roster"
for yr in year_list:
    new_url = f"{base_url}/{yr}"
    url_list.append(new_url)
    

df_list = []
for url in url_list:
    if "2021" not in url:
        data = pd.read_html(url)[2]
        df_list.append(data.rename(columns = {"Year":"Class",
                                             "Academic Year":"Class",
                                             "Pos":"Pos.",
                                             "Full Name": "Name"}))

for year_num in range(len(year_list)-1):
    df_list[year_num]["year"] = year_list[year_num]

c_w_soccer = split(pd.concat(df_list, join = "outer", ignore_index = True)).drop(
    columns = ["Hometown / High School","#"])
c_w_soccer["sport"] = "soccer"
c_womens.append(c_w_soccer)
        
    
c_womens_df = pd.concat(c_womens, join = "outer", ignore_index = True)
c_womens_df



,Name,Pos.,Ht.,Class,School,year,Hometown,High School,sport
0,Susie Rafiu,Forward,6-1,So.,CC,2010,"Ipswich, England",Copleston,basketball
1,Perri Page,Guard/Forward,5-11,So.,CC,2010,"Pittsburgh, Pa.",Chartiers Valley,basketball
2,Ava Sciolla,Guard,6-0,So.,CC,2010,"Fairless Hills, Pa.",Pennsbury,basketball
3,Cecelia Collins,Guard,6-0,Jr.,CC,2010,"Scranton, Pa.",Scanton Prep,basketball
4,Fliss Henderson,Guard/Forward,5-11,FY,CC,2010,"North Curl Curl, Australia",St. Luke's Grammar,basketball
...,...,...,...,...,...,...,...,...,...
1625,Kendall McBride,F,5-7,Sr.,CC,2022,"Falls Church, Va.",Bishop O'Connell,soccer
1626,Ola Weber,M,5-11,Sr.,BC,2022,"Warren, N.J.",The Pingry School,soccer
1627,Claire Pinnie,F,5-4,Jr.,SEAS,2022,"West Chester, Pa.",Academy of Notre Dame de Namur,soccer
1628,Spencer Robbins,M/F,5-10,Jr.,SEAS,2022,"Nashville, Tenn.",Harpeth Hall School,soccer


In [113]:
#combined Columbia
c_mens_df["sex"] = "mens"
c_womens_df["sex"] = "womens"

c_both = [c_mens_df, c_womens_df]
pd.concat(c_both, join = "outer", ignore_index = True)

,Name,Pos.,Ht.,Class,year,Wt.,School,Hometown,High School,sport,Previous Club,Event,sex
0,Paul Delaney,P,6-3,FY,2010,NaN,NaN,"Palatine, Ill.",Loyola Academy,football,NaN,NaN,mens
1,Mike Stephens,WR,5-11,Sr.,2010,NaN,NaN,"Flower Mound, Texas",Flower Mound,football,NaN,NaN,mens
2,Brian DeVeau,WR,5-11,So.,2010,NaN,NaN,"Kinnelon, N.J.",Kinnelon,football,NaN,NaN,mens
3,Nico Gutierrez,WR,6-0,Sr.,2010,NaN,NaN,"New Canaan, Conn.",New Canaan,football,NaN,NaN,mens
4,Ryan Murphy,LB,6-2,So.,2010,NaN,NaN,"San Antonio, Texas",Alamo Heights,football,NaN,NaN,mens
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4457,Kendall McBride,F,5-7,Sr.,2022,NaN,CC,"Falls Church, Va.",Bishop O'Connell,soccer,NaN,NaN,womens
4458,Ola Weber,M,5-11,Sr.,2022,NaN,BC,"Warren, N.J.",The Pingry School,soccer,NaN,NaN,womens
4459,Claire Pinnie,F,5-4,Jr.,2022,NaN,SEAS,"West Chester, Pa.",Academy of Notre Dame de Namur,soccer,NaN,NaN,womens
4460,Spencer Robbins,M/F,5-10,Jr.,2022,NaN,SEAS,"Nashville, Tenn.",Harpeth Hall School,soccer,NaN,NaN,womens
